In [6]:
# import module
from geopy.geocoders import Nominatim
import string
import pandas as pd
import re, os
from geopy.point import Point


In [7]:
CURRENT_DIR = os.getcwd()
PARENT_DIR = CURRENT_DIR.replace("/notebooks", "")

RAW_DATA_DIR = os.path.join(PARENT_DIR ,"data/raw")
CLEAN_DATA_DIR = os.path.join(PARENT_DIR, "data/preprocessed/clean")


In [8]:
# pick the most recent file
df = pd.read_csv(f'{CLEAN_DATA_DIR}/clean_eq_2022_08_20.csv')
df = df.drop("Unnamed: 0",axis=1)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   p_id               9 non-null      int64  
 1   date               9 non-null      object 
 2   time               9 non-null      object 
 3   location           9 non-null      object 
 4   filtered_location  9 non-null      object 
 5   lat                9 non-null      float64
 6   long               9 non-null      float64
 7   magnitude          9 non-null      float64
dtypes: float64(3), int64(1), object(4)
memory usage: 704.0+ bytes


In [10]:
geolocator = Nominatim(user_agent="located_coord")

In [11]:
lat = "-34.34751"
long = "-70.5792"
location = geolocator.reverse(Point(lat, long))
print(location)

Requínoa, Provincia de Cachapoal, Región del Libertador General Bernardo O'Higgins, Chile


In [12]:
location.raw['address']

{'town': 'Requínoa',
 'county': 'Provincia de Cachapoal',
 'state': "Región del Libertador General Bernardo O'Higgins",
 'ISO3166-2-lvl4': 'CL-LI',
 'country': 'Chile',
 'country_code': 'cl'}

In [13]:
df['lat'] = df['lat'].astype(str)
df['long'] = df['long'].astype(str)

In [14]:
def _get_location_info(point):
    location = geolocator.reverse(point)
    if location is None:
        return None
    return location.raw['address']
location_df = df[['id', 'lat','long', 'filtered_location']]

KeyError: "['id'] not in index"

In [ ]:
location_df['geom_data'] = location_df["lat"].map(str)  + ',' + location_df['long'].map(str)
location_df['loc_info'] = location_df['geom_data'].apply(lambda x : _get_location_info(x))

/var/folders/9k/93y5mwqx7vv2w2hhhthftg980000gn/T/ipykernel_74483/1144252590.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  location_df['geom_data'] = location_df["lat"].map(str)  + ',' + location_df['long'].map(str)
/var/folders/9k/93y5mwqx7vv2w2hhhthftg980000gn/T/ipykernel_74483/1144252590.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  location_df['loc_info'] = location_df['geom_data'].apply(lambda x : _get_location_info(x))


In [ ]:
def _extract_loc_info(address):
    if address is not None:
        country = address.get('country', '')
    else:
        country = None
    return country


In [ ]:
location_df['country'] = location_df['loc_info'].apply(lambda x : _extract_loc_info(x))

/var/folders/9k/93y5mwqx7vv2w2hhhthftg980000gn/T/ipykernel_74483/1984943729.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  location_df['country'] = location_df['loc_info'].apply(lambda x : _extract_loc_info(x))


In [ ]:
location_df

,lat,long,filtered_location,geom_data,loc_info,country
0,-57.9983,-25.7068,South Atlantic Ocean,"-57.9983,-25.7068",None,None
1,-20.5094,-11.7191,South Atlantic Ocean,"-20.5094,-11.7191",None,None
2,-56.1713,-27.3639,South Atlantic Ocean,"-56.1713,-27.3639","{'state': 'South Sandwich Islands', 'country':...",South Georgia and the South Sandwich Islands
3,-20.4496,-11.7901,South Atlantic Ocean,"-20.4496,-11.7901",None,None
4,-20.2238,-12.4116,South Atlantic Ocean,"-20.2238,-12.4116",None,None
5,-22.2898,171.6659,South Pacific Ocean,"-22.2898,171.6659",None,None
6,0.3403,124.3752,Molucca Sea,"0.3403,124.3752","{'state': 'Sulawesi Utara', 'ISO3166-2-lvl4': ...",Indonesia
7,37.6284,141.6741,North Pacific Ocean,"37.6284,141.6741",None,None
8,-34.3475,-70.5794,Chile,"-34.3475,-70.5794","{'town': 'Requínoa', 'county': 'Provincia de C...",Chile


In [ ]:
location_df['country'].fillna(location_df['filtered_location'], inplace=True)

/var/folders/9k/93y5mwqx7vv2w2hhhthftg980000gn/T/ipykernel_74483/253824053.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  location_df['country'].fillna(location_df['filtered_location'], inplace=True)


In [ ]:
location_df= location_df[['id','lat', 'long', 'country', 'geom_data']]

In [ ]:
location_df

,lat,long,filtered_location,geom_data,loc_info,country
0,-57.9983,-25.7068,South Atlantic Ocean,"-57.9983,-25.7068",None,South Atlantic Ocean
1,-20.5094,-11.7191,South Atlantic Ocean,"-20.5094,-11.7191",None,South Atlantic Ocean
2,-56.1713,-27.3639,South Atlantic Ocean,"-56.1713,-27.3639","{'state': 'South Sandwich Islands', 'country':...",South Georgia and the South Sandwich Islands
3,-20.4496,-11.7901,South Atlantic Ocean,"-20.4496,-11.7901",None,South Atlantic Ocean
4,-20.2238,-12.4116,South Atlantic Ocean,"-20.2238,-12.4116",None,South Atlantic Ocean
5,-22.2898,171.6659,South Pacific Ocean,"-22.2898,171.6659",None,South Pacific Ocean
6,0.3403,124.3752,Molucca Sea,"0.3403,124.3752","{'state': 'Sulawesi Utara', 'ISO3166-2-lvl4': ...",Indonesia
7,37.6284,141.6741,North Pacific Ocean,"37.6284,141.6741",None,North Pacific Ocean
8,-34.3475,-70.5794,Chile,"-34.3475,-70.5794","{'town': 'Requínoa', 'county': 'Provincia de C...",Chile
